## 간단한 자연어 분석/처리 예제



In [28]:
# colab 환경 기본 설치된 패키지
!pip install spacy

In [29]:
import spacy

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

In [30]:
# load model
nlp = spacy.load('en_core_web_sm')

"""
문장을 구성하는 요소들을 쪼개 token을 만들고 token에는 다음과 같은 변수들이 정의된다.

text - 원본 text
lemma - 단어의 기본 형태
pos - 간단한 POS(part-of-speech) 태그
tag - 자세한 POS 태그
dep - 구문적 종속성(토큰 간의 관계)
shape - 단어의 모양(대문자, 구두점, 숫자)
is_alpha - 토큰이 alpha만 있는가(True/False)
is_stop - is, the, of, on, at등 내용적 의미가 없는 단어인가(True/False)
"""
text = ("When Sebastian Thrun started working on self-driving cars at "
        "Google in 2007, few people outside of the company took him ")
doc = nlp(text)

str_format = "{:>10}"*8
print(str_format.format("text", "lemma", "pos", "tag", "dep", "shape", "is_alpha", "is_stop"))
print("=="*40)
for token in doc:
  print(str_format.format(token.text, token.lemma_, token.pos_, token.tag_,
                            token.dep_, token.shape_, str(token.is_alpha), str(token.is_stop)))

      text     lemma       pos       tag       dep     shape  is_alpha   is_stop
      When      when     SCONJ       WRB    advmod      Xxxx      True      True
 Sebastian sebastian       ADJ        JJ      amod     Xxxxx      True     False
     Thrun     Thrun     PROPN       NNP     nsubj     Xxxxx      True     False
   started     start      VERB       VBD     advcl      xxxx      True     False
   working      work      VERB       VBG     xcomp      xxxx      True     False
        on        on       ADP        IN      prep        xx      True      True
      self      self      NOUN        NN  npadvmod      xxxx      True     False
         -         -     PUNCT      HYPH     punct         -     False     False
   driving     drive      VERB       VBG      amod      xxxx      True     False
      cars       car      NOUN       NNS      pobj      xxxx      True     False
        at        at       ADP        IN      prep        xx      True      True
    Google    Google     PRO

In [31]:
"""
docs.ents - 의미를 이해한 토큰을 반환한다.(사람인지, 조직인지 날짜인지 등)
"""
for entity in doc.ents:
    print(entity.text, entity.label_)

Sebastian Thrun PERSON
Google ORG
2007 DATE


In [32]:
"""
CountVectorizer
텍스트 데이터를 분석할 때 사용할 수 있는 전처리 중 하나이다.
텍스트를 단어들의 카운트로 벡터화한다.
"""
sample = [
  "The drug Aspirin inhibits the protein COX-2.",
  "The drug Ibuprofen targets the protein COX-1.",
  "The drug Corona kill the protein COX-1."
]

vectorizer = CountVectorizer()
vectorized_sample = vectorizer.fit_transform(sample)
print(vectorized_sample.toarray())

# 각 단어가 몇번째 열에 해당하는지 볼 수 있다.
print(vectorizer.vocabulary_)

[[1 0 1 1 0 1 0 1 0 2]
 [0 0 1 1 1 0 0 1 1 2]
 [0 1 1 1 0 0 1 1 0 2]]
{'the': 9, 'drug': 3, 'aspirin': 0, 'inhibits': 5, 'protein': 7, 'cox': 2, 'ibuprofen': 4, 'targets': 8, 'corona': 1, 'kill': 6}


In [33]:
"""
※ 머신러닝 알고리즘의 자세한 내용은 교재 뒤편에 등장합니다. 가볍게 지나가주세요.

vectorizer한 텍스트는 다음과 같이 모델 학습에 사용될 수 있다.
train_data의 3가지 문장과 라벨은 아래와 같습니다.

"아스피린이라는 약물은 COX-2 단백질을 억제합니다.", 억제한다.
"이부프로펜이라는 약물은 COX-1 단백질을 강화합니다.", 강화한다.
"코로나라는 약물은 박테리아 COX-1을 약화시킵니다.", 약화시킨다.

3가지 문장을 학습하고
"코로나는 COX-2 단백질에 영향을 미칩니다." 라는 문장을 predict하면 "약화시킨다" 라는 label로 분류합니다.
"""
nlp = spacy.load("en_core_web_sm")

label = ["inhibits", "strengthens", "weaken"]
train_data = [
    "The drug Aspirin inhibits the protein COX-2.",
    "The drug ibuprofen strengthens the protein COX-1.",
    "The drug Corona weaken the bacteria COX-1."
]

vectorizer = CountVectorizer()
vectorized_train_data = vectorizer.fit_transform(train_data)

model = RandomForestClassifier()
model.fit(vectorized_train_data, label)

vectorized_test_data = vectorizer.transform(["The corona affects the protein COX-2."])
result = model.predict(vectorized_test_data)
print(result)

['weaken']
